In [85]:
import numpy as np
import pandas as pd

data = pd.read_csv("income.csv")

In [86]:
data

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income_>50K
0,67,Private,366425,Doctorate,16,Divorced,Exec-managerial,Not-in-family,White,Male,99999,0,60,United-States,1
1,17,Private,244602,12th,8,Never-married,Other-service,Own-child,White,Male,0,0,15,United-States,0
2,31,Private,174201,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
3,58,State-gov,110199,7th-8th,4,Married-civ-spouse,Transport-moving,Husband,White,Male,0,0,40,United-States,0
4,25,State-gov,149248,Some-college,10,Never-married,Other-service,Not-in-family,Black,Male,0,0,40,United-States,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43952,52,Private,68982,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,50,United-States,1
43953,19,Private,116562,HS-grad,9,Never-married,Other-service,Own-child,White,Female,0,0,40,United-States,0
43954,30,Private,197947,Some-college,10,Divorced,Sales,Not-in-family,White,Male,0,0,58,United-States,0
43955,46,Private,97883,Bachelors,13,Never-married,Sales,Not-in-family,White,Female,0,0,35,United-States,0


In [87]:
# Menghapus baris dengan data duplikat
data.drop_duplicates(inplace=True)

In [88]:
data.dropna(inplace=True)

In [89]:
import numpy as np

for column in data.select_dtypes(exclude=[np.number]).columns: 
    data = pd.concat([data, pd.get_dummies(data[column], prefix=column, drop_first=True)], axis=1)
    data.drop(column, axis=1, inplace=True)

In [90]:
X = data.drop('income_>50K', axis=1)
y = data['income_>50K']

In [91]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [92]:
from sklearn.preprocessing import MinMaxScaler

mm = MinMaxScaler()
X_train = mm.fit_transform(X_train)
X_test = mm.transform(X_test)

# TabNet Classifier

In [93]:
# Menggunakan pustaka TabNet

from pytorch_tabnet.tab_model import TabNetClassifier

clf = TabNetClassifier()

clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], patience=10, max_epochs=20)

/Users/vallentaustin/pyenv/env/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cpu
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 0.53581 | val_0_auc: 0.79733 | val_1_auc: 0.80285 |  0:00:03s
epoch 1  | loss: 0.43243 | val_0_auc: 0.7868  | val_1_auc: 0.79794 |  0:00:06s
epoch 2  | loss: 0.39786 | val_0_auc: 0.861   | val_1_auc: 0.87073 |  0:00:09s
epoch 3  | loss: 0.37781 | val_0_auc: 0.85748 | val_1_auc: 0.86633 |  0:00:11s
epoch 4  | loss: 0.36391 | val_0_auc: 0.85525 | val_1_auc: 0.86274 |  0:00:14s
epoch 5  | loss: 0.3525  | val_0_auc: 0.87235 | val_1_auc: 0.87739 |  0:00:17s
epoch 6  | loss: 0.3454  | val_0_auc: 0.88485 | val_1_auc: 0.89053 |  0:00:20s
epoch 7  | loss: 0.34669 | val_0_auc: 0.8899  | val_1_auc: 0.89708 |  0:00:23s
epoch 8  | loss: 0.34251 | val_0_auc: 0.89185 | val_1_auc: 0.89823 |  0:00:25s
epoch 9  | loss: 0.33843 | val_0_auc: 0.89653 | val_1_auc: 0.90291 |  0:00:28s
epoch 10 | loss: 0.33363 | val_0_auc: 0.89917 | val_1_auc: 0.90412 |  0:00:31s
epoch 11 | loss: 0.33386 | val_0_auc: 0.89982 | val_1_auc: 0.90395 |  0:00:34s
epoch 12 | loss: 0.33222 | val_0_auc: 0.90251 | val_

/Users/vallentaustin/pyenv/env/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


In [94]:
from sklearn.metrics import accuracy_score, classification_report

predictions = clf.predict(X_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.85


In [95]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.95      0.91      6137
           1       0.77      0.57      0.66      2001

    accuracy                           0.85      8138
   macro avg       0.82      0.76      0.78      8138
weighted avg       0.85      0.85      0.85      8138



In [96]:
prediction = clf.predict(X_test)
comparison_df = pd.DataFrame({'True Label': y_test, 'Predicted Label': prediction})
print(comparison_df.head(10)) 

       True Label  Predicted Label
14093           0                0
43864           0                0
30514           1                0
4709            0                0
26857           0                0
1497            0                0
15002           0                0
11034           0                0
2682            0                0
21779           0                1
